# Example use of the Julia module Sonic.Linckii

Note that the module depends on another module ```LinckiiSecrets``` for authentication, which is not part of the Git repository, and has to be added by hand.

In [ ]:
import Pkg                      # package manager
Pkg.activate("..")              # my Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html
include("../src/Sonic.jl")      # my project

In [ ]:
import JuliaDB, Plots, Random

In [ ]:
site_name = "keab"

GET "https://canary.noda.se/$site_name/api/v1/node"

In [ ]:
@time nodes = Sonic.Linckii.get_nodes(site_name)

Flatten ```nodes```, i.e.,
```
flat_nodes = JuliaDB.rename(
    JuliaDB.flatten(nodes, :sensor_ids),
    :sensor_ids => :sensor_id,
)
```

In [ ]:
flat_nodes = Sonic.Linckii.flatten_nodes(nodes)

GET "https://canary.noda.se/$site_name/api/v1/device"

In [ ]:
@time devices = Sonic.Linckii.get_devices(site_name)

GET "https://canary.noda.se/$site_name/api/v1/sensor"

In [ ]:
@time sensors = Sonic.Linckii.get_sensors(site_name)

Get device sensors,

In [ ]:
device_sensors = JuliaDB.reindex(
    JuliaDB.join(
        devices,
        sensors;
        lkey = :protocol_id,
        rkey = :protocol_id,
    ),
    (:device_id, :sensor_id),
)

Get node device sensors,

In [ ]:
node_device_sensors = JuliaDB.reindex(
    JuliaDB.join(
        flat_nodes,
        device_sensors;
        lkey = (:device_id, :sensor_id),
        rkey = (:device_id, :sensor_id),
    ),
    (:node_id, :sensor_id),
)

Get side 1 substation data for 2019-09-01/2019-10-01 and 2019-10-01/2019-11-01,

In [ ]:
pkeys = JuliaDB.select(
    JuliaDB.join(
        Sonic.Linckii.flatten_nodes(
            JuliaDB.filter(
                r -> (
                    # r.node_id <= 20 && # uncomment to speed things up
                    r.device_id == 4
                ),
                nodes,
            ),
        ),
        JuliaDB.filter(
            r -> r.sensor_name in [
                :meter_effect,
                :meter_primreturntemp,
                :meter_primsupplytemp,
                :meter_volumeflow,
            ],
            device_sensors,
        );
        lkey = (:device_id, :sensor_id),
        rkey = (:device_id, :sensor_id),
    ),
    (:node_id, :sensor_name),
)

In [ ]:
dates = ["2019-09-01", "2019-10-01", "2019-11-01"]

Save data,

In [ ]:
# @time JuliaDB.save(Sonic.Linckii.get_data(site_name, pkeys, dates...), "Example Sonic.Linckii.db")

Load data,

In [ ]:
@time data = JuliaDB.load("Example Sonic.Linckii.db")

Simple plot,

In [ ]:
node_id = JuliaDB.select(data, :node_id)[Random.rand(1 : end)]

In [ ]:
sensor_name = JuliaDB.select(data, :sensor_name)[Random.rand(1 : end)]

In [ ]:
t = Sonic.Linckii.filter(r -> r.node_id == node_id && r.sensor_name == sensor_name, data)
x = JuliaDB.select(t, :datetime)
y = JuliaDB.select(t, :value)
Plots.plot(x, y, xlabel = :datetime, label = sensor_name)